# Import Library

In [1]:
import os
import csv
import pandas as pd
import shutil
os.chdir('/mnt/c/Workspace/Ch3/wflow/')
os.getcwd()

'/mnt/c/Workspace/Ch3/wflow'

# Temp directory

In [2]:
wsName_list = ["stark_ohio_19389766_03118500",
          "san-diego_california_20331196_11023340",
          "bexar_texas_10836388_08181480",
          #"douglas_georgia_3286256_02337000",
          "fulton_georgia_2047963_02336410"
         ]

# For test purpose, set wsName.
wsName = "fulton_georgia_2047963_02336410"


In [25]:
model_directory = str("../wflow_models/" + wsName + "/")
intbl_directory = str("../wflow_models/" + wsName + "/intbl/")
static_directory = str("../wflow_models/" +wsName + "/staticmaps/")
temp_directory = str("../wflow_models/" + wsName + "/temp/")
csv_file_path = str("../wflow_models/combined_samples.csv")
if not os.path.exists(temp_directory):
    os.mkdir(temp_directory)

In [38]:
df = pd.read_csv(csv_file_path)[0:20]
print(df)

    beta_W  beta_HP   D_H   D_P  Kf_fast  Kf_slow  Sumax_W  Sumax_HP
0     2.99     3.16  0.04  0.04    0.048    0.001    142.0     253.0
1     3.94     4.08  0.06  0.08    0.051    0.004     52.0      59.0
2     1.94     2.13  0.08  0.09    0.043    0.003    202.0     223.0
3     2.45     2.52  0.25  0.35    0.037    0.002     43.0      68.0
4     4.05     4.28  0.45  0.47    0.049    0.004     82.0      95.0
5     2.25     2.46  0.24  0.28    0.054    0.002    216.0     239.0
6     4.51     4.73  0.15  0.22    0.051    0.001    171.0     319.0
7     2.14     2.14  0.39  0.40    0.061    0.008     39.0      70.0
8     4.78     4.79  0.16  0.19    0.039    0.003    210.0     413.0
9     2.61     2.61  0.57  0.64    0.063    0.007    188.0     346.0
10    1.41     1.53  0.47  0.58    0.069    0.002    209.0     219.0
11    2.37     2.40  0.06  0.08    0.044    0.009    213.0     267.0
12    3.38     3.40  0.37  0.44    0.042    0.009     49.0      57.0
13    2.47     2.57  0.03  0.03   

# Define some functions

In [22]:
import re

def modify_and_save_intbl(file_path, new_number, new_file_path):
    # Read the existing text
    with open(file_path, 'r', encoding='utf-8') as file:
        existing_text = file.read()
    
    # Use regular expression to find and replace the last number in the given pattern
    pattern = r'(\[0,> \[0,> \[0,> )\d+\.?\d*'
    replacement = r'\g<1>' + str(new_number)  # Convert new_number to string if not already
    modified_text = re.sub(pattern, replacement, existing_text)
    
    # Save the modified text to the new file path
    with open(new_file_path, 'w', encoding='utf-8') as new_file:
        new_file.write(modified_text)

In [21]:
import pcraster as pcr
import numpy as np

def process_Kf_files(directory, identifiers, static_directory):
    """
    Process files in the specified directory with the given identifiers.
    
    Parameters:
    - directory: The path to the directory containing the files.
    - identifiers: A list of identifiers for the files to be processed.
    """
    TIA =  pcr.readmap(static_directory + "/TIA.map")


    for i in identifiers:          
        percent = pcr.readmap(static_directory + "/wflow_percent" + i + ".map")
        catchment = pcr.readmap(static_directory + "/wflow_ws.map")

        TIA_stat = pcr.areaaverage(TIA * percent, catchment)
        TIA_stat2 = pcr.pcr2numpy(TIA_stat, mv=np.nan)
        TIA_stat3 = np.nanmean(TIA_stat2)
        
        #print(str("Average TIA for " + i + " :"+ str(TIA_stat3)))
        
        if i == "W":
            #print("For W, not performing Kf modification")
            continue
                        
        # Construct the original and new file names
        original_file_name = f"Kf{i}_OG.tbl"
        new_file_name = f"Kf{i}.tbl"
        
        # Path to the original and new files
        original_file_path = os.path.join(directory, original_file_name)
        new_file_path = os.path.join(directory, new_file_name)
        
        # Open the original file
        with open(original_file_path, 'r') as file:
            # Read the line
            line = file.readline().strip()
            # Extract the last number
            parts = line.split(' ')
            last_number = float(parts[-1])
            
            # Perform the calculation
            new_number = round(last_number * (1 + 0.02/2 * TIA_stat3 * 100), 4)
            #new_number = last_number
            
            # Prepare the new line with the calculated number
            new_line = line.rsplit(' ', 1)[0] + f' {new_number}'
        
        # Write the new line to the new file
        with open(new_file_path, 'w') as file:
            file.write(new_line)


In [31]:
def modify_and_save_ini(file_path, index, new_file_path):
    # Read the existing content from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Modify lines containing 'intbl'
    modified_lines = []
    for line in lines:
        if 'intbl' in line:
            modified_line = f'intbl = temp/{index}/intbl\n'
            modified_lines.append(modified_line)
        else:
            modified_lines.append(line)
    
    # Save the modified content to a new file
    with open(new_file_path, 'w', encoding='utf-8') as new_file:
        new_file.writelines(modified_lines)

# Loop

In [40]:
print(df.iterrows())

<generator object DataFrame.iterrows at 0x7fd7e9047ca0>


In [37]:
identifiers = ['W', 'H', 'P']

for index, row in df.iterrows():
    
    print(f"\r{index + 1} / {len(df)}", end='    ', flush=True)
    temp_iter_directory = str(temp_directory+str(index))
    temp_iter_intbl_directory = temp_iter_directory + "/intbl/"
    
    #if not os.path.exists(temp_iter_directory):
    #    os.mkdir(temp_iter_directory)
        
    try:
        shutil.copytree(intbl_directory, temp_iter_intbl_directory, dirs_exist_ok=True)
    except Exception as e:
        print(f"Error occurred: {e}")
        
    modify_and_save_ini(model_directory+"/wflow_tofuflex.ini", index, temp_iter_directory+"/wflow_tofuflex_"+str(index)+".ini")
        
    modify_and_save_intbl(temp_iter_intbl_directory+"/betaW"+".tbl", row[0], temp_iter_intbl_directory+"/betaW"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/betaAP"+".tbl", row[0], temp_iter_intbl_directory+"/betaAP"+".tbl")
    
    modify_and_save_intbl(temp_iter_intbl_directory+"/betaH"+".tbl", row[1], temp_iter_intbl_directory+"/betaH"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/betaP"+".tbl", row[1], temp_iter_intbl_directory+"/betaP"+".tbl")
    
    modify_and_save_intbl(temp_iter_intbl_directory+"/DH"+".tbl", row[2], temp_iter_intbl_directory+"/DH"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/DP"+".tbl", row[3], temp_iter_intbl_directory+"/DP"+".tbl")
    
    modify_and_save_intbl(temp_iter_intbl_directory+"/KfW_OG"+".tbl", row[4], temp_iter_intbl_directory+"/KfW_OG"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/KfaP_OG"+".tbl", row[4], temp_iter_intbl_directory+"/KfaP_OG"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/Kfimp_OG"+".tbl", row[4], temp_iter_intbl_directory+"/Kfimp_OG"+".tbl")
    
    modify_and_save_intbl(temp_iter_intbl_directory+"/KfP_OG"+".tbl", row[5], temp_iter_intbl_directory+"/KfP_OG"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/KfH_OG"+".tbl", row[5], temp_iter_intbl_directory+"/KfH_OG"+".tbl")

    modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxW"+".tbl", row[6], temp_iter_intbl_directory+"/sumaxW"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxH"+".tbl", row[7], temp_iter_intbl_directory+"/sumaxH"+".tbl")
    modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxP"+".tbl", row[7], temp_iter_intbl_directory+"/sumaxP"+".tbl")
    
    process_Kf_files(temp_iter_intbl_directory, identifiers, static_directory)

print()

20 / 20    
